In [40]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
import logging
import time

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper

# 高相似多少
coum=5
# 加载保存的数组
lin = np.load("lin.npy")
# lin = np.linspace(0,3,401)
# lin = np.linspace(0.01,1,100)


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
cells_name = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx', header=0).columns[2:] # 保证cells的名字从第一行的第三个开始
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [41]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [42]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [43]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [44]:
# 分离x y数据构造字典
dict_columns_split = {} # 高相似字典
# 计算x数据
x = data2.drop(columns=columns)  # 其他所有的列
for i in columns:
    # b 要预测协同得分那一列
    y = data2.loc[:, i]
    # A 权重
    x_i = x.copy()
    x_i['y'] = y

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index = tp)
    df.sort_values(by=0, inplace=True, ascending=True)
    max_l = df.index[-coum:]
    max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

    # 存储 对应数据
    count = 0 # 第几行 int
    for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=y[count]
        dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
        count += 1

In [45]:
dict_columns_split["'SUNITINIB', 'MK-4827',32"]['max_x_line']

array([-0.14583069,  0.17624582,  0.40102622, -0.4231079 ,  0.06873453])

In [46]:
TTT = [] # 值列表
cct = 0
NNN = []
for i in Tanimoto.index: 
    y = list(Tanimoto.iloc[:,cct])
    y.pop(cct) 
    tp_columns = list(columns.copy())
    tp_columns.pop(cct)
    df = pd.DataFrame(y, index=tp_columns)
    df.sort_values(by=0 , inplace=True, ascending=True) 
    max_l = df.index[-coum:]
    NNN.extend(max_l)
    # 计算max_5y
    max_5y = []
    for j in max_l:
        max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
    # 将max_5y加入TTT
    TTT += max_5y
    # 修改cct变量的值
    cct += 1

In [47]:
def alpha_pcc(t_coum, a, lin, linename, dict_columns_split, TTT, coum, data2, NNN):
    alpha = []
    result_pcc = []
    pcc_split = []
    pre_dict = []
    ori_dict = []
    for alp in lin:
        pre = []
        for elem in range(39):
            pp = [] 
            if TTT[(coum * a - t_coum):(coum * a)] == []:
                print("ERROR-TTT")
            for j in TTT[(coum * a - t_coum):(coum * a)]: 
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
            pp = np.array(pp)  # 将 pp 转换为 numpy 数组
            pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
        pre = np.array(pre)  # 将 pre 转换为 numpy 数组
        pre_dict.append(pre)
        ori_dict.append(data2[linename])
        pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    alpha.append(lin[pcc_split.index(np.max(pcc_split))])
    pre = pre_dict[pcc_split.index(np.max(pcc_split))]
    ori = ori_dict[pcc_split.index(np.max(pcc_split))]
    pcc=result_pcc[0]
    temp_NNN = NNN[(coum * a - t_coum):(coum * a)]
    temp_TTT = TTT[(coum * a - t_coum):(coum * a)]
    del result_pcc, pcc_split, pp
    return alpha, pcc, temp_NNN, temp_TTT, pre, ori

def alp_mse_rmse(alpha, t_coum, a, linename, dict_columns_split, TTT, coum):
    r1 = []  # 初始化 r1 为空列表
    alp = alpha[0]
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]: 
            try:
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
            except:
                print(linename)
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
    r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
    mse = np.mean(np.square(r1))
    rmse = math.sqrt(mse)
    del r1, pp,
    return alp, mse, rmse

In [48]:
# 单进程 分函数
def calculate_parameters(t_coum, coum, TTT, data2, dict_columns_split, lin, NNN):
    a = 1
    Name_data = []
    temp_NNN_list, temp_TTT_list, pre_list, ori_list = [], [], [], []
    for linename in columns:  
        # 求PCC
        alpha, pcc, temp_NNN, temp_TTT, pre, ori = alpha_pcc(t_coum, a, lin, linename, dict_columns_split, TTT, coum, data2, NNN)
        # 求MSE
        alp, mse, rmse = alp_mse_rmse(alpha, t_coum, a, linename, dict_columns_split, TTT, coum)
        Name_data.append([linename, t_coum, alp, mse, pcc, rmse])
        temp_NNN_list.append(temp_NNN)
        temp_TTT_list.append(temp_TTT)
        pre_list.append(pre)
        ori_list.append(ori)
        a = a+1
    pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])["pcc"].mean()
    print(pcc_mean)
    return [pcc_mean, Name_data, temp_NNN_list, temp_TTT_list, pre_list, ori_list]

In [49]:
# 多进程
@timeit
def run_parallel(columns, lin,NNN):
    with multiprocessing.Pool() as pool:
        tp = []
        for t_coum in range(2, coum+1, 1):
            tp.append((t_coum, coum, TTT, data2, dict_columns_split, lin, NNN,))
        results = pool.starmap(calculate_parameters, tp)
    return results

PCC_mean = 0
res_pcc_list = []
results = run_parallel(columns,lin,NNN)
sorted_list = sorted(results, key=lambda x: x[0])
max_res = sorted_list[-1][1]
print("MAX PCC: ", sorted_list[-1][0])

0.3134730948109902
0.3508114189090811
0.33450193291801794
0.35616993203738073
run_parallel took 22.469946 seconds
MAX PCC:  0.35616993203738073


In [51]:
# 将结果保存到 CSV 文件中
pd.DataFrame(max_res).to_csv('pcc_cpu_test_multi_unite_coum.csv', header=None, index=False)
rebuild_list = []
for i,j,k in zip(max_res, sorted_list[-1][2], sorted_list[-1][3]):
        for q,l in zip(j,k):
            rebuild_list.append([q,i[0],l,i[1],i[2],i[4],i[5]])
pd.DataFrame(rebuild_list,columns=["target_neighbor","target","similarity","neighbors","alp","pcc","rmse"]).to_csv("pcc_cpu_test_multi_target_unite_coum.csv")
temp_list = []
for i,j,k in zip(max_res, sorted_list[-1][4], sorted_list[-1][5]):
        for m,q,l in zip(cells_name,j,k):
            temp_list.append([i[0],m,q,l])
pd.DataFrame(temp_list,columns=["drugs","cells_line","pridict","origin"]).sort_values("cells_line").to_csv("pcc_cpu_test_multi_cells_line_unite_coum.csv")

In [ ]:
# 单线程
# # 加载保存的数组
# lin = np.load("lin.npy")
# PCC_mean = 0
# res_pcc_list = []
# for t_coum in range(2, coum+1, 1):    #邻居
#     PCC = 0
#     Name_data = []
#     a = 1
#     for linename in columns:
#         #求PCC
#         alpha = []
#         result_pcc = []
#         pcc_split = []
#         for alp in lin:
#             # r1 = []
#             # ct = 0
#             pre = []
#             for elem in range(39):
#                 pp = []
#                 for j in TTT[(coum * a - t_coum):(coum * a)]: 
#                     pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))#改函数
#                 pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
#             pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#         result_pcc.append(np.max(pcc_split))
#         alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#         pcc=result_pcc[0]

#         #求MSE
#         r1 = []
#         ct = 0
#         alp = alpha[0]
#         for elem in range(39):
#             pp = []
#             for j in TTT[(coum * a - t_coum):(coum * a)]:
#                 pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  # 改函数
#             r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) -
#                             dict_columns_split[f"{linename},{elem}"]['y_line'])
#         mse=np.mean(np.square(r1))
#         rmse=math.sqrt(mse)
#         Name_data.append([linename, t_coum, alp, mse, pcc, rmse])
#         a = a+1
#     pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"])["pcc"].mean()
#     if pcc_mean > PCC_mean:
#         PCC_mean = pcc_mean
#         res_pcc_list = Name_data
# pcc_mean = pd.DataFrame(res_pcc_list, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse"]).to_csv('pcc_unite.csv', header=None, index=False)